In [1]:
!pip install PyGreSQL
# !pip install nbimporter

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import csv
import pgdb
import pickle
import zipfile
# import nbimporter
import numpy as np
import pandas as pd
import scipy
# import ProviderPredictionScript

from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputClassifier

In [3]:
### How many providers do we want to synthesize?

## User defined for each specialty
num_providers = 1000

In [4]:
### Probability of number of claims per provider

### We'll eventually bring inference (of number of claims for each provider) from all customers for each specialty.

### Based on MonteCarlo simulation from the empirical data
num_claims_df = pd.read_csv('num_claims_df.csv')


num_iter = 100000
num_claims_avg_lst = []
for ii in range(0, num_iter):
    num_claims_avg_lst.append(round(num_claims_df.sample(round(0.1*len(num_claims_df))).num_claims.mean()))
dist_mean = np.round(np.mean(num_claims_avg_lst))
dist_std = np.round(np.std(num_claims_avg_lst))
group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
scipy.stats.describe(group)

DescribeResult(nobs=1000, minmax=(49, 231), mean=134.661, variance=923.5456246246247, skewness=0.038752944155496356, kurtosis=-0.056853546854489956)

In [5]:
group

[98,
 140,
 107,
 83,
 121,
 146,
 139,
 134,
 100,
 134,
 128,
 114,
 94,
 138,
 168,
 132,
 192,
 94,
 118,
 95,
 97,
 98,
 168,
 113,
 138,
 84,
 136,
 82,
 157,
 113,
 111,
 183,
 134,
 141,
 142,
 122,
 128,
 130,
 154,
 87,
 123,
 116,
 131,
 156,
 50,
 78,
 182,
 111,
 144,
 141,
 124,
 185,
 108,
 90,
 161,
 106,
 87,
 120,
 179,
 152,
 182,
 109,
 120,
 108,
 117,
 157,
 182,
 148,
 94,
 148,
 163,
 98,
 189,
 113,
 136,
 130,
 169,
 110,
 96,
 184,
 139,
 162,
 161,
 94,
 117,
 84,
 124,
 131,
 148,
 124,
 102,
 115,
 151,
 141,
 93,
 77,
 108,
 126,
 153,
 99,
 77,
 140,
 118,
 148,
 131,
 159,
 187,
 126,
 138,
 169,
 188,
 146,
 154,
 133,
 111,
 129,
 148,
 123,
 134,
 164,
 111,
 166,
 103,
 107,
 145,
 106,
 166,
 150,
 125,
 192,
 137,
 164,
 171,
 229,
 128,
 151,
 123,
 120,
 140,
 186,
 131,
 113,
 156,
 158,
 171,
 158,
 137,
 137,
 171,
 117,
 134,
 168,
 78,
 111,
 106,
 107,
 113,
 183,
 129,
 189,
 124,
 162,
 110,
 134,
 143,
 154,
 212,
 151,
 138,
 114,
 132

In [6]:
### Determine probability of different combinations 
### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
all_combinations = pd.read_csv('all_combinations.csv')
sel_combinations = all_combinations[all_combinations.prob_claims>=0.01]

In [7]:
## Unique combinations and corresponding weights
comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
weights = sel_combinations['prob_claims'].tolist()

In [8]:
weights

[0.01996026935043309,
 0.011276493185901465,
 0.02120915557762814,
 0.01465798045602606,
 0.019405208805013074,
 0.01882093454667621,
 0.010845590920378026,
 0.014285505616336309,
 0.04288573056192577,
 0.013029315960912051,
 0.011093907480171193,
 0.015746191262178466,
 0.026883919311724926,
 0.012335490279137029,
 0.011123121193088037,
 0.02031083390543521,
 0.031112604256437974,
 0.036772761134076334,
 0.0955726618074524,
 0.11689867223674795,
 0.022772089218679247,
 0.018207446575422503,
 0.011816946874863059,
 0.016169790099472694,
 0.012240545712157288,
 0.016082148960722163,
 0.022940068067951097,
 0.012963585106849154,
 0.015234951286133709,
 0.010115248097456946,
 0.028132805538919968,
 0.04085537751420517,
 0.025313682242444603,
 0.052847606666569284,
 0.06675333401498662,
 0.010297833803187216,
 0.010495026365375908,
 0.010348957800791692,
 0.02717605644089336,
 0.03816041249762639,
 0.010458509224229851,
 0.01311695709966258,
 0.019317567666262542,
 0.02289624749857583,
 0.

In [9]:
import random  
final_df_list = []
random_outliers = random.sample(range(0, num_providers), int(round(0.01*num_providers)))
for ii in range(0, num_providers):
    if ii in random_outliers:
        tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    else:
        tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    tmp['ren_id'] = 'prov_' + str(ii)
    tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
    final_df_list.append(tmp)
final_df = pd.concat(final_df_list)

In [10]:
final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id
0,90837,22,F,F4322,WD@95,prov_0,claim_0
1,90837,22,M,F4320,WD@GT,prov_0,claim_1
2,90837,22,F,F411,WD@95,prov_0,claim_2
3,90837,30,None,F4325,WD,prov_0,claim_3
4,90837,30,F,F4323,WD,prov_0,claim_4
...,...,...,...,...,...,...,...
134,90834,30,None,F411,WD,prov_999,claim_134
135,90837,22,F,F411,WD,prov_999,claim_135
136,90837,30,F,F331,WD,prov_999,claim_136
137,90837,30,F,F332,WD,prov_999,claim_137


In [11]:
final_df[final_df['ren_id'] == 'prov_0']

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id
0,90837,22,F,F4322,WD@95,prov_0,claim_0
1,90837,22,M,F4320,WD@GT,prov_0,claim_1
2,90837,22,F,F411,WD@95,prov_0,claim_2
3,90837,30,None,F4325,WD,prov_0,claim_3
4,90837,30,F,F4323,WD,prov_0,claim_4
...,...,...,...,...,...,...,...
93,90837,22,F,F411,WD@GT,prov_0,claim_93
94,90837,30,None,F411,WD,prov_0,claim_94
95,90837,30,None,F411,WD,prov_0,claim_95
96,90837,30,None,F411,WD,prov_0,claim_96


In [12]:
final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id
0,90837,22,F,F4322,WD@95,prov_0,claim_0
1,90837,22,M,F4320,WD@GT,prov_0,claim_1
2,90837,22,F,F411,WD@95,prov_0,claim_2
3,90837,30,None,F4325,WD,prov_0,claim_3
4,90837,30,F,F4323,WD,prov_0,claim_4
...,...,...,...,...,...,...,...
134,90834,30,None,F411,WD,prov_999,claim_134
135,90837,22,F,F411,WD,prov_999,claim_135
136,90837,30,F,F331,WD,prov_999,claim_136
137,90837,30,F,F332,WD,prov_999,claim_137


providers area is alphereta with further away having fewer paitients<br>
<b>work on cost </b><br>
send me a sample members file ecuding phi using age to randomize the date

In [13]:
a = []
for index, row in final_df.iterrows():  
    b = (row['proc_values'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)
    
final_df["combo"] = a
    


In [14]:
final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id,combo
0,90837,22,F,F4322,WD@95,prov_0,claim_0,90837@95@WD
1,90837,22,M,F4320,WD@GT,prov_0,claim_1,90837@GT@WD
2,90837,22,F,F411,WD@95,prov_0,claim_2,90837@95@WD
3,90837,30,None,F4325,WD,prov_0,claim_3,90837@WD
4,90837,30,F,F4323,WD,prov_0,claim_4,90837@WD
...,...,...,...,...,...,...,...,...
134,90834,30,None,F411,WD,prov_999,claim_134,90834@WD
135,90837,22,F,F411,WD,prov_999,claim_135,90837@WD
136,90837,30,F,F331,WD,prov_999,claim_136,90837@WD
137,90837,30,F,F332,WD,prov_999,claim_137,90837@WD


In [15]:
cost = pd.read_csv("amt_df.csv")

cost

,proc_values,diag_values,mod_values,place_of_service,pat_gender,amount_charged,amount_paid
0,90834@90834,F4322,WD,30,None,100.0,94.95
1,90837,F4325,WD,30,None,110.0,67.28
2,90837,F4310,WD,30,None,110.0,89.70
3,90837@90837@90791@90837@90837@90837,F329,WD,30,None,100.0,89.70
4,90837@90837@90837@90837@90837@90837@90837@9083...,F341,WD,30,None,130.0,89.70
...,...,...,...,...,...,...,...
1370057,90837@90837@90837@90837@90837@90837@90837@9083...,F341,WD,30,M,150.0,0.00
1370058,90837@90837,F4312,WD@GT,22,F,120.0,109.00
1370059,90837,F419,WD,30,M,175.0,0.00
1370060,90834@90834@90834@90834@90834@90834@90834@9083...,F411,WD@95:WD@95:WD@95,22,F,300.0,0.00


In [ ]:
a = []
for index, row in cost.iterrows():  
    b = (row['proc_values'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)
    
cost["combo"] = a
cost = cost.drop(columns=['diag_values', 'place_of_service',"pat_gender", "mod_values", "proc_values"])
cost.head()


In [ ]:
c = cost.groupby(['combo'])['amount_charged'].sum()#["combo"].size()
c = c.reset_index()
e = cost.groupby(['combo'])['amount_charged'].size()
e = e.reset_index()
c["count"] = e["amount_charged"]
c["avg_charge"] = c["amount_charged"]/ c["count"]


d = cost.groupby(['combo'])['amount_paid'].sum()#["combo"].size()
d = d.reset_index()
e = cost.groupby(['combo'])['amount_paid'].size()
e = e.reset_index()
d["count"] = e["amount_paid"]
d["avg_paid_pct"] = (d["amount_paid"]/ d["count"])/c["amount_charged"]
#d.to_csv("d.csv")
d

In [ ]:
c

In [ ]:
scb = 50#may want to change scale so that it is more in line with reality
sca = 4

cost_dict = cost.set_index('combo').to_dict()
final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["amount_charged"][x])
final_df['real charge'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
final_df = final_df.drop(columns=['cost'])



paid_dict = c.set_index('combo').to_dict()
#final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["amount_charged"][x])
final_df['real paid'] = np.random.normal(final_df['real charge']*.8, scale = sca).astype(int)
#final_df = final_df.drop(columns=['cost'])
final_df
final_df
final_df.to_csv("final.csv")
final_df